In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# import string library for text preprocessing
import string

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv('../data/merged/imdb_movies_detail_complete.csv')

In [4]:
df.head()

,image_link,imdb_id,rating_count,best_rating,worst_rating,rating_x,genre,keywords,trailer_link,actors,director,synopsis,summary,id,title,year,runtime,rating_y,votes,plot
0,https://m.media-amazon.com/images/M/MV5BMDliMm...,tt0172495,1610899,10,1,8.5,"['Action', 'Adventure', 'Drama']","['roman empire', 'gladiator', 'slavery', 'comb...",https://www.imdb.com/video/vi2628367897/,"['Russell Crowe', 'Joaquin Phoenix', 'Connie N...",['Ridley Scott'],"Shouting ""Roma Invicta!"" as his forces attack,...",NaN,tt0172495,1. Gladiator,2000,2000,8.5 (1.6M),(1.6M),A former Roman General sets out to exact venge...
1,https://m.media-amazon.com/images/M/MV5BZTcyNj...,tt0209144,1316937,10,1,8.4,"['Mystery', 'Thriller']","['memory', 'short term memory loss', 'hidden t...",https://www.imdb.com/video/vi3220356889/,"['Guy Pearce', 'Carrie-Anne Moss', 'Joe Pantol...",['Christopher Nolan'],"Leonard Shelby (Guy Pearce), a man whose short...",NaN,tt0209144,2. Memento,2000,2000,8.4 (1.3M),(1.3M),A man with short-term memory loss attempts to ...
2,https://m.media-amazon.com/images/M/MV5BMTA2ND...,tt0208092,906115,10,1,8.2,"['Comedy', 'Crime']","['diamond', 'boxer', 'narrated by character', ...",https://www.imdb.com/video/vi1558577433/,"['Jason Statham', 'Brad Pitt', 'Stephen Graham']",['Guy Ritchie'],After stealing an 86-carat (17 g) diamond in a...,NaN,tt0208092,3. Snatch,2000,2000,8.2 (906K),(906K),"Unscrupulous boxing promoters, violent bookmak..."
3,https://m.media-amazon.com/images/M/MV5BZTM2ZG...,tt0144084,709394,10,1,7.6,"['Crime', 'Drama', 'Horror']","['narcissism', 'materialism', 'serial murder',...",https://www.imdb.com/video/vi4060743449/,"['Christian Bale', 'Justin Theroux', 'Josh Luc...",['Mary Harron'],A white background. Red drops begin to fall pa...,NaN,tt0144084,4. American Psycho,2000,2000,7.6 (709K),(709K),A wealthy New York City investment banking exe...
4,https://m.media-amazon.com/images/M/MV5BNjUxYz...,tt0187393,292416,10,1,7.2,"['Action', 'Drama', 'History']","['american revolution', '18th century', 'briti...",https://www.imdb.com/video/vi100139289/,"['Mel Gibson', 'Heath Ledger', 'Joely Richards...",['Roland Emmerich'],"Benjamin Martin (Mel Gibson), a veteran of the...",NaN,tt0187393,5. The Patriot,2000,2000,7.2 (292K),(292K),Peaceful farmer Benjamin Martin is driven to l...


In [5]:
df.shape

(2400, 20)

In [6]:
df.columns

Index(['image_link', 'imdb_id', 'rating_count', 'best_rating', 'worst_rating',
       'rating_x', 'genre', 'keywords', 'trailer_link', 'actors', 'director',
       'synopsis', 'summary', 'id', 'title', 'year', 'runtime', 'rating_y',
       'votes', 'plot'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   image_link    2400 non-null   object 
 1   imdb_id       2400 non-null   object 
 2   rating_count  2400 non-null   int64  
 3   best_rating   2400 non-null   int64  
 4   worst_rating  2400 non-null   int64  
 5   rating_x      2400 non-null   float64
 6   genre         2400 non-null   object 
 7   keywords      2400 non-null   object 
 8   trailer_link  2199 non-null   object 
 9   actors        2400 non-null   object 
 10  director      2400 non-null   object 
 11  synopsis      1222 non-null   object 
 12  summary       1886 non-null   object 
 13  id            2400 non-null   object 
 14  title         2400 non-null   object 
 15  year          2400 non-null   int64  
 16  runtime       2400 non-null   int64  
 17  rating_y      2400 non-null   object 
 18  votes         2400 non-null 

In [8]:
df.nunique()

image_link      2400
imdb_id         2400
rating_count    2277
best_rating        1
worst_rating       1
rating_x          22
genre            289
keywords        2358
trailer_link    2198
actors          2368
director        1638
synopsis        1222
summary         1886
id              2400
title           2400
year              25
runtime           25
rating_y        1633
votes            604
plot            2400
dtype: int64

In [9]:
for col in df.columns:
    print(col)
    print(df[col].unique())

image_link
['https://m.media-amazon.com/images/M/MV5BMDliMmNhNDEtODUyOS00MjNlLTgxODEtN2U3NzIxMGVkZTA1L2ltYWdlXkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_.jpg'
 'https://m.media-amazon.com/images/M/MV5BZTcyNjk1MjgtOWI3Mi00YzQwLWI5MTktMzY4ZmI2NDAyNzYzXkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_.jpg'
 'https://m.media-amazon.com/images/M/MV5BMTA2NDYxOGYtYjU1Mi00Y2QzLTgxMTQtMWI1MGI0ZGQ5MmU4XkEyXkFqcGdeQXVyNDk3NzU2MTQ@._V1_.jpg'
 ...
 'https://m.media-amazon.com/images/M/MV5BYzhkYjdlYzAtZGEwNy00NmUzLTg3NzEtOTFmNjJmZWJiMTU3XkEyXkFqcGdeQXVyNzc0MTgzMzU@._V1_.jpg'
 'https://m.media-amazon.com/images/M/MV5BYmY2YmEyYTItYjA4Yy00MTNmLTg1ZDgtNjFlM2QzMDI4MTM2XkEyXkFqcGdeQXVyMjM4NTM5NDY@._V1_.jpg'
 'https://m.media-amazon.com/images/M/MV5BZTFjYTEwOTEtYTk2OS00MzU3LTg0ZmUtOGE4MzljNmQzZGE5XkEyXkFqcGdeQXVyMDc5ODIzMw@@._V1_.jpg']
imdb_id
['tt0172495' 'tt0209144' 'tt0208092' ... 'tt14351082' 'tt4996328'
 'tt30796448']
rating_count
[1610899 1316937  906115 ...    3574    2453   10027]
best_rating
[10]
worst_rating
[1]
rating_x
[8

In [10]:
df.isnull().sum()

image_link         0
imdb_id            0
rating_count       0
best_rating        0
worst_rating       0
rating_x           0
genre              0
keywords           0
trailer_link     201
actors             0
director           0
synopsis        1178
summary          514
id                 0
title              0
year               0
runtime            0
rating_y           0
votes              0
plot               0
dtype: int64

In [11]:
# update column plot with synopsis. If synopsis is not available, use summary. If both are null, use existing plot
df['plot'] = df['synopsis'].fillna('') + df['summary'].fillna('') 
df['plot'] = df['plot'].fillna(df['plot'])

In [12]:
df['plot'].isna().sum()

0

In [13]:
df.isnull().sum()

image_link         0
imdb_id            0
rating_count       0
best_rating        0
worst_rating       0
rating_x           0
genre              0
keywords           0
trailer_link     201
actors             0
director           0
synopsis        1178
summary          514
id                 0
title              0
year               0
runtime            0
rating_y           0
votes              0
plot               0
dtype: int64

In [14]:
df[df['trailer_link'].isnull()].head(5)

,image_link,imdb_id,rating_count,best_rating,worst_rating,rating_x,genre,keywords,trailer_link,actors,director,synopsis,summary,id,title,year,runtime,rating_y,votes,plot
41,https://m.media-amazon.com/images/M/MV5BMTQ4NT...,tt0264578,2186,10,1,7.8,"['Documentary', 'History']","['reenactment', 'expedition', 'antarctica', 'w...",NaN,"['Liam Neeson', 'Julian Ayer', 'John Blackborow']",['George Butler'],"The Endurance, 2002Donald M HayesAugust 1, 191...",NaN,tt0264578,43. The Endurance,2000,2000,7.8 (2.2K),(2.2K),"The Endurance, 2002Donald M HayesAugust 1, 191..."
43,https://m.media-amazon.com/images/M/MV5BYWExZT...,tt0118852,4620,10,1,7.1,['Drama'],"['artist', 'screenplay adapted by author', 'po...",NaN,"['Al Pacino', 'Jerry Orbach', 'Susan Floyd']",['Al Pacino'],Harry Levine (Al Pacino) is a washed-up writer...,NaN,tt0118852,45. Chinese Coffee,2000,2000,7.1 (4.6K),(4.6K),Harry Levine (Al Pacino) is a washed-up writer...
44,https://m.media-amazon.com/images/M/MV5BMGQ4OD...,tt0262240,1446,10,1,8.2,"['Documentary', 'Biography']","['found footage', 'home movie', 'experimental ...",NaN,"['Jonas Mekas', 'Stan Brakhage', 'Robert Breer']",['Jonas Mekas'],NaN,NaN,tt0262240,46. As I Was Moving Ahead Occasionally I Saw B...,2000,2000,8.2 (1.4K),(1.4K),
53,https://m.media-amazon.com/images/M/MV5BMjAxNz...,tt0240641,1769,10,1,7.0,['Documentary'],"['dysfunctional family', 'sexual abuse', 'gran...",NaN,"['James Ronald Whitney', 'Melvin Just', 'Denis...",['James Ronald Whitney'],NaN,NaN,tt0240641,"55. Just, Melvin: Just Evil",2000,2000,7.0 (1.8K),(1.8K),
54,https://m.media-amazon.com/images/M/MV5BNDI5Zj...,tt0236216,6188,10,1,7.6,"['Documentary', 'Biography', 'Music']","['music history', 'rock &apos;n&apos; roll', '...",NaN,"['Paul Cook', 'Steve Jones', 'John Lydon']",['Julien Temple'],NaN,NaN,tt0236216,56. The Filth and the Fury,2000,2000,7.6 (6.2K),(6.2K),


In [15]:
df['trailer_link'] = df['trailer_link'].fillna('No trailer available')

In [16]:
# check if runtime and year column has same values
df[df['year'] != df['runtime']]

,image_link,imdb_id,rating_count,best_rating,worst_rating,rating_x,genre,keywords,trailer_link,actors,director,synopsis,summary,id,title,year,runtime,rating_y,votes,plot


In [17]:
# drop unwanted columns
df.drop(columns=['synopsis', 'summary', 'best_rating', 'worst_rating', 'rating_y', 'runtime', 'votes', 'id'], inplace=True)

In [18]:
df.head()

,image_link,imdb_id,rating_count,rating_x,genre,keywords,trailer_link,actors,director,title,year,plot
0,https://m.media-amazon.com/images/M/MV5BMDliMm...,tt0172495,1610899,8.5,"['Action', 'Adventure', 'Drama']","['roman empire', 'gladiator', 'slavery', 'comb...",https://www.imdb.com/video/vi2628367897/,"['Russell Crowe', 'Joaquin Phoenix', 'Connie N...",['Ridley Scott'],1. Gladiator,2000,"Shouting ""Roma Invicta!"" as his forces attack,..."
1,https://m.media-amazon.com/images/M/MV5BZTcyNj...,tt0209144,1316937,8.4,"['Mystery', 'Thriller']","['memory', 'short term memory loss', 'hidden t...",https://www.imdb.com/video/vi3220356889/,"['Guy Pearce', 'Carrie-Anne Moss', 'Joe Pantol...",['Christopher Nolan'],2. Memento,2000,"Leonard Shelby (Guy Pearce), a man whose short..."
2,https://m.media-amazon.com/images/M/MV5BMTA2ND...,tt0208092,906115,8.2,"['Comedy', 'Crime']","['diamond', 'boxer', 'narrated by character', ...",https://www.imdb.com/video/vi1558577433/,"['Jason Statham', 'Brad Pitt', 'Stephen Graham']",['Guy Ritchie'],3. Snatch,2000,After stealing an 86-carat (17 g) diamond in a...
3,https://m.media-amazon.com/images/M/MV5BZTM2ZG...,tt0144084,709394,7.6,"['Crime', 'Drama', 'Horror']","['narcissism', 'materialism', 'serial murder',...",https://www.imdb.com/video/vi4060743449/,"['Christian Bale', 'Justin Theroux', 'Josh Luc...",['Mary Harron'],4. American Psycho,2000,A white background. Red drops begin to fall pa...
4,https://m.media-amazon.com/images/M/MV5BNjUxYz...,tt0187393,292416,7.2,"['Action', 'Drama', 'History']","['american revolution', '18th century', 'briti...",https://www.imdb.com/video/vi100139289/,"['Mel Gibson', 'Heath Ledger', 'Joely Richards...",['Roland Emmerich'],5. The Patriot,2000,"Benjamin Martin (Mel Gibson), a veteran of the..."


In [19]:
df.columns

Index(['image_link', 'imdb_id', 'rating_count', 'rating_x', 'genre',
       'keywords', 'trailer_link', 'actors', 'director', 'title', 'year',
       'plot'],
      dtype='object')

In [20]:
# renamed rating_x to rating
df.rename(columns={'rating_x': 'rating'}, inplace=True)

In [21]:
df.columns

Index(['image_link', 'imdb_id', 'rating_count', 'rating', 'genre', 'keywords',
       'trailer_link', 'actors', 'director', 'title', 'year', 'plot'],
      dtype='object')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   image_link    2400 non-null   object 
 1   imdb_id       2400 non-null   object 
 2   rating_count  2400 non-null   int64  
 3   rating        2400 non-null   float64
 4   genre         2400 non-null   object 
 5   keywords      2400 non-null   object 
 6   trailer_link  2400 non-null   object 
 7   actors        2400 non-null   object 
 8   director      2400 non-null   object 
 9   title         2400 non-null   object 
 10  year          2400 non-null   int64  
 11  plot          2400 non-null   object 
dtypes: float64(1), int64(2), object(9)
memory usage: 225.1+ KB


In [23]:
df.isnull().sum()

image_link      0
imdb_id         0
rating_count    0
rating          0
genre           0
keywords        0
trailer_link    0
actors          0
director        0
title           0
year            0
plot            0
dtype: int64

# Preprocess plot

In [24]:
# Convert text to lowercase
def convert_to_lower_case(text):
  return text.lower()

In [25]:
def remove_punctuation(text):
  # Create a translation table to remove punctuation
  translator = str.maketrans('', '', string.punctuation)

  # Remove punctuation using the translation table
  text_without_punct = text.translate(translator)

  return text_without_punct

In [26]:
def tokenize(text):
  # Tokenization
  return word_tokenize(text)

In [27]:
# Remove stopwords
def remove_stop_words(tokens):
  stop_words = set(stopwords.words('english'))
  return [word for word in tokens if word not in stop_words]

In [28]:
def lemmatize(tokens):
  # Lemmatization
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(word) if lemmatizer.lemmatize(word) is not None else word for word in tokens]

In [29]:
# Remove special characters and numbers
def remove_special_chars(tokens):
    return [re.sub('[^A-Za-z]+', '', word) for word in tokens]

In [30]:
# function to do preprocessing text
def preprocess_text(text):
  lower_text = convert_to_lower_case(text) # converting to lowercase
  removed_punctuation = remove_punctuation(lower_text) # removing punctuation
  tokens = tokenize(removed_punctuation) # tokenize words
  tokens = remove_stop_words(tokens) # removing stop words
  tokens = lemmatize(tokens) # lemmatize tokens
  tokens = remove_special_chars(tokens) # remove special characters from token

  preprocessed_text = ' '.join(tokens) # final preprocessed text
  return preprocessed_text

In [31]:
preprocess_text("Hi!, my name is 123......testing the code noww11111")

'hi name testing code noww'

In [32]:
# Apply function on text column
df['processed_plot'] = df['plot'].apply(preprocess_text)

In [33]:
df['plot'].isnull().sum()

0

In [34]:
df['title']

0                                            1. Gladiator
1                                              2. Memento
2                                               3. Snatch
3                                      4. American Psycho
4                                          5. The Patriot
5                                  6. Requiem for a Dream
6                                           7. Sexy Beast
7                                      8. Erin Brockovich
8                                                9. X-Men
9                                              10. Malena
10                                11. Remember the Titans
11                                      12. Almost Famous
12                                          13. Cast Away
13                         14. O Brother, Where Art Thou?
14                           15. The Emperor's New Groove
15                                        16. Unbreakable
16                                        17. Chicken Run
17            

In [35]:
# remove numbers from title
df['title'] = df['title'].apply(lambda x: re.sub(r'[\d+.]', '', x))
df['title']

0                                               Gladiator
1                                                 Memento
2                                                  Snatch
3                                         American Psycho
4                                             The Patriot
5                                     Requiem for a Dream
6                                              Sexy Beast
7                                         Erin Brockovich
8                                                   X-Men
9                                                  Malena
10                                    Remember the Titans
11                                          Almost Famous
12                                              Cast Away
13                             O Brother, Where Art Thou?
14                               The Emperor's New Groove
15                                            Unbreakable
16                                            Chicken Run
17            

In [36]:
# remove single apostrophe from columns keywords, actors, director, genre
def remove_apostrophe(text):
  return re.sub(r"\'", '', text)

list_cols = ['keywords', 'actors', 'director', 'genre']
for col in list_cols:
    df[col] = df[col].apply(remove_apostrophe)

In [37]:
df.head()

,image_link,imdb_id,rating_count,rating,genre,keywords,trailer_link,actors,director,title,year,plot,processed_plot
0,https://m.media-amazon.com/images/M/MV5BMDliMm...,tt0172495,1610899,8.5,"[Action, Adventure, Drama]","[roman empire, gladiator, slavery, combat, bat...",https://www.imdb.com/video/vi2628367897/,"[Russell Crowe, Joaquin Phoenix, Connie Nielsen]",[Ridley Scott],Gladiator,2000,"Shouting ""Roma Invicta!"" as his forces attack,...",shouting rom invicta force attack general maxi...
1,https://m.media-amazon.com/images/M/MV5BZTcyNj...,tt0209144,1316937,8.4,"[Mystery, Thriller]","[memory, short term memory loss, hidden truth,...",https://www.imdb.com/video/vi3220356889/,"[Guy Pearce, Carrie-Anne Moss, Joe Pantoliano]",[Christopher Nolan],Memento,2000,"Leonard Shelby (Guy Pearce), a man whose short...",leonard shelby guy pearce man whose short term...
2,https://m.media-amazon.com/images/M/MV5BMTA2ND...,tt0208092,906115,8.2,"[Comedy, Crime]","[diamond, boxer, narrated by character, cockne...",https://www.imdb.com/video/vi1558577433/,"[Jason Statham, Brad Pitt, Stephen Graham]",[Guy Ritchie],Snatch,2000,After stealing an 86-carat (17 g) diamond in a...,stealing carat g diamond heist antwerp franki...
3,https://m.media-amazon.com/images/M/MV5BZTM2ZG...,tt0144084,709394,7.6,"[Crime, Drama, Horror]","[narcissism, materialism, serial murder, socio...",https://www.imdb.com/video/vi4060743449/,"[Christian Bale, Justin Theroux, Josh Lucas]",[Mary Harron],American Psycho,2000,A white background. Red drops begin to fall pa...,white background red drop begin fall past open...
4,https://m.media-amazon.com/images/M/MV5BNjUxYz...,tt0187393,292416,7.2,"[Action, Drama, History]","[american revolution, 18th century, british ar...",https://www.imdb.com/video/vi100139289/,"[Mel Gibson, Heath Ledger, Joely Richardson]",[Roland Emmerich],The Patriot,2000,"Benjamin Martin (Mel Gibson), a veteran of the...",benjamin martin mel gibson veteran french ind...


In [47]:
df.to_csv('../data/processed/preprocessed_dataset.csv', index=False)

## Using TF-IDF

In [44]:
model = train_model(processed_df['processed_plot'], tf_idf_vectorizer)

NameError: name 'processed_df' is not defined

In [ ]:
find_similar_plot(test_txt, tf_idf_vectorizer, model).iloc[0]['Plot Summary']

"After the kind-hearted but clumsy Super Dave Osborne (Bob Einstein) survives a near-fatal accident, he discovers his manager has embezzled from him. This forces him into bankruptcy and causes him to lose his house. Super Dave decides to retire from stunt work, but ends up meeting a single mother named Sandy (Gia Carides). When Super Dave learns that Sandy's son, Timmy (Carl Michael Lindner) needs an expensive heart surgery, Super Dave comes out of retirement to raise the money. In the process, he must face off against his arch-enemy, Gil Ruston (Dan Hedaya).\n"

In [ ]:
find_similar_plot(text_txt1, tf_idf_vectorizer, model).iloc[0]['Plot Summary']

"After the kind-hearted but clumsy Super Dave Osborne (Bob Einstein) survives a near-fatal accident, he discovers his manager has embezzled from him. This forces him into bankruptcy and causes him to lose his house. Super Dave decides to retire from stunt work, but ends up meeting a single mother named Sandy (Gia Carides). When Super Dave learns that Sandy's son, Timmy (Carl Michael Lindner) needs an expensive heart surgery, Super Dave comes out of retirement to raise the money. In the process, he must face off against his arch-enemy, Gil Ruston (Dan Hedaya).\n"

## Using Bag of Words

In [ ]:
model = train_model(processed_df['ProcessedPlot'], bow_vectorizer)

C:\Users\msani\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [ ]:
find_similar_plot(test_txt, bow_vectorizer, model).iloc[0]['Plot Summary']

"After the kind-hearted but clumsy Super Dave Osborne (Bob Einstein) survives a near-fatal accident, he discovers his manager has embezzled from him. This forces him into bankruptcy and causes him to lose his house. Super Dave decides to retire from stunt work, but ends up meeting a single mother named Sandy (Gia Carides). When Super Dave learns that Sandy's son, Timmy (Carl Michael Lindner) needs an expensive heart surgery, Super Dave comes out of retirement to raise the money. In the process, he must face off against his arch-enemy, Gil Ruston (Dan Hedaya).\n"

In [ ]:
find_similar_plot(text_txt1, tf_idf_vectorizer, model).iloc[0]['Plot Summary']

"After the kind-hearted but clumsy Super Dave Osborne (Bob Einstein) survives a near-fatal accident, he discovers his manager has embezzled from him. This forces him into bankruptcy and causes him to lose his house. Super Dave decides to retire from stunt work, but ends up meeting a single mother named Sandy (Gia Carides). When Super Dave learns that Sandy's son, Timmy (Carl Michael Lindner) needs an expensive heart surgery, Super Dave comes out of retirement to raise the money. In the process, he must face off against his arch-enemy, Gil Ruston (Dan Hedaya).\n"

## Testing

In [ ]:
processed_df.iloc[-1]['Plot Summary']

'In London, Marc enjoys a Christmas party with friends and Oliver, his husband of many years. Oliver leaves the party early to travel for a book signing in Paris and dies in a car accident just seconds after.\nIn the following year, Marc\'s best friends Sophie and Thomas try to help him through his grief. However, Marc opens a note from Oliver confessing he had met someone else. Then he discovers his husband had also been secretly leasing a flat in Paris.\nMarc brings Sophie and Thomas with him to the city, though he keeps the revelation a secret so the three can have a good time. Sophie, who had recently ended a serious relationship with her boyfriend Terrance, brings them to meet a date, which Marc leaves to spend time with Theo, a French man he had previously met at a London performance art exhibit.\nMarc admits that his anger at Oliver\'s secrets is distracting him from his grief. He also reveals that he gave up painting following the death of his mother, and he worries that he\'s 

In [ ]:
test_txt2 = ["In London, Marc enjoys a Christmas party with friends and Oliver, his husband of many years. Oliver leaves the party early to travel for a book signing in Paris and dies in a car accident just seconds after.\nIn the following year, Marc\'s best friends Sophie and Thomas try to help him through his grief. However, Marc opens a note from Oliver confessing he had met someone else. Then he discovers his husband had also been secretly leasing a flat in Paris.\nMarc brings Sophie and Thomas with him to the city, though he keeps the revelation a secret so the three can have a good time. Sophie, who had recently ended a serious relationship with her boyfriend Terrance, brings them to meet a date, which Marc leaves to spend time with Theo, a French man he had previously met at a London performance art exhibit.\nMarc admits that his anger at Oliver\'s secrets is distracting him from his grief. He also reveals that he gave up painting following the death of his mother, and he worries that he\'s not properly grieving Oliver just as he didn\'t properly grieve her. Theo brings Marc to the Claude Monet room at the Musée de l\'Orangerie, explaining that it had been painted after the death of his family, and they kiss.\nThe next morning, Thomas berates the two for their abandonment, explaining that Sophie had been found sleeping at a bus station and he could not reach Marc. A man enters and introduces himself as Luca, a Berlin native who had won Oliver\'s art scholarship before growing closer to him, finally leading to Marc telling the truth about the nature of their trip."]

In [ ]:
find_similar_plot(test_txt2, bow_vectorizer, model).iloc[0]['Plot Summary']

"After the kind-hearted but clumsy Super Dave Osborne (Bob Einstein) survives a near-fatal accident, he discovers his manager has embezzled from him. This forces him into bankruptcy and causes him to lose his house. Super Dave decides to retire from stunt work, but ends up meeting a single mother named Sandy (Gia Carides). When Super Dave learns that Sandy's son, Timmy (Carl Michael Lindner) needs an expensive heart surgery, Super Dave comes out of retirement to raise the money. In the process, he must face off against his arch-enemy, Gil Ruston (Dan Hedaya).\n"